# Week 9: Single-Agent Systems - SOLUTION

## Overview
This notebook contains complete implementations for Week 9: Single-Agent Systems.

### Learning Objectives
By the end of this week, you will be able to:
- Design and implement agent architecture with planner, memory, and tools
- Implement tool calling mechanisms for agents
- Build short-term and long-term memory systems
- Handle errors and implement retry logic for robust agents
- Create autonomous task-solving agents

### Real-World Outcome
Build an **Autonomous Research Agent** that can independently gather information, synthesize findings, and produce comprehensive research reports.

---

## Part 1: Agent Architecture Fundamentals - SOLUTION

In [ ]:
from typing import List, Dict, Optional, Any
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum

class AgentStatus(Enum):
    """Agent execution status."""
    IDLE = "idle"
    PLANNING = "planning"
    EXECUTING = "executing"
    REFLECTING = "reflecting"
    COMPLETED = "completed"
    FAILED = "failed"

@dataclass
class AgentMemory:
    """Agent memory storage."""
    short_term: List[Dict[str, Any]] = field(default_factory=list)
    long_term: Dict[str, Any] = field(default_factory=dict)
    working_memory: Dict[str, Any] = field(default_factory=dict)
    max_short_term_size: int = 10
    
    def add_to_short_term(self, entry: Dict[str, Any]):
        """Add entry to short-term memory with timestamp."""
        entry['timestamp'] = datetime.now().isoformat()
        self.short_term.append(entry)
        
        # Limit short-term memory size
        if len(self.short_term) > self.max_short_term_size:
            self.short_term.pop(0)
    
    def store_long_term(self, key: str, value: Any):
        """Store information in long-term memory."""
        self.long_term[key] = {
            'value': value,
            'stored_at': datetime.now().isoformat()
        }
    
    def retrieve(self, query: str) -> Optional[Any]:
        """Retrieve relevant information from memory."""
        # Check long-term memory first
        if query in self.long_term:
            return self.long_term[query]['value']
        
        # Search short-term memory
        for entry in reversed(self.short_term):
            if query in str(entry):
                return entry
        
        return None

@dataclass
class AgentTask:
    """Represents a task for the agent."""
    description: str
    steps: List[str] = field(default_factory=list)
    status: AgentStatus = AgentStatus.IDLE
    result: Optional[str] = None
    created_at: datetime = field(default_factory=datetime.now)

class Agent:
    """Base autonomous agent."""
    
    def __init__(self, name: str, system_prompt: str):
        self.name = name
        self.system_prompt = system_prompt
        self.memory = AgentMemory()
        self.tools: Dict[str, Dict[str, Any]] = {}
        self.status = AgentStatus.IDLE
        self.max_iterations = 10
    
    def register_tool(self, name: str, function: callable, description: str):
        """Register a tool that the agent can use."""
        self.tools[name] = {
            'function': function,
            'description': description
        }
        print(f"Tool registered: {name}")
    
    def plan(self, task: AgentTask) -> List[str]:
        """Break down task into actionable steps."""
        self.status = AgentStatus.PLANNING
        
        # Simple planning logic (in production, use LLM)
        steps = [
            f"Analyze task: {task.description}",
            "Identify required tools",
            "Execute task steps",
            "Synthesize results",
            "Return final output"
        ]
        
        task.steps = steps
        self.memory.add_to_short_term({
            'type': 'planning',
            'task': task.description,
            'steps': steps
        })
        
        return steps
    
    def execute(self, task: AgentTask) -> str:
        """Execute the agent's task."""
        print(f"Agent {self.name} starting task: {task.description}")
        
        # Plan the task
        steps = self.plan(task)
        print(f"Plan created with {len(steps)} steps")
        
        # Execute with iteration limit
        self.status = AgentStatus.EXECUTING
        results = []
        
        for i, step in enumerate(steps):
            if i >= self.max_iterations:
                break
            
            print(f"Executing step {i+1}: {step}")
            results.append(f"Completed: {step}")
            
            self.memory.add_to_short_term({
                'type': 'execution',
                'step': step,
                'result': results[-1]
            })
        
        # Generate final result
        final_result = f"Task '{task.description}' completed with {len(results)} steps"
        task.result = final_result
        task.status = AgentStatus.COMPLETED
        self.status = AgentStatus.COMPLETED
        
        return final_result

# Test the implementation
agent = Agent("ResearchAgent", "You are a research assistant.")
print(f"Agent created: {agent.name}")
print(f"Status: {agent.status}")

# Test task execution
task = AgentTask(description="Research quantum computing")
result = agent.execute(task)
print(f"\nTask result: {result}")
print(f"Agent status: {agent.status}")